# Исследование надежности заемщиков

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')

Выведем первые строчки датафрейма data на экран и изучим основную информацию.

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:** 
В данных наблюдаются пропуски, далее необходимо сделать предобработку данных. Выяснить существуют ли дубликаты, неправильные типы данных, аномалии.

## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [8]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

Артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`

In [9]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()
data = data.drop_duplicates()

## Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных 

### Зависимость между количеством детей и возвратом кредита в срок

In [21]:
data_grouped_children = data.groupby('children').agg({'debt': ['count','mean']})
data_grouped_children

debt          
          count      mean
children                 
0         14091  0.075438
1          4808  0.092346
2          2052  0.094542
3           330  0.081818
4            41  0.097561
5             9  0.000000

**Вывод:** 
    
   Из полученных данных можно сделать вывод, что чем больше детей, тем выше процент количества людей с кредитом. 
    
   Однако в существующих данных есть аномалия при количестве детей равных 5, при этом количестве детей у людей нет кредитов. 
    Это может быть связано с недостатком данных, может указывать на порядочность людей или на отказах в выдаче кредитов семьям с большим количеством детей.

### Зависимость между семейным положением и возвратом кредита в срок

In [22]:
data_grouped_family_status = data.groupby('family_status').agg({'debt': ['count','mean']})
data_grouped_family_status

debt          
                       count      mean
family_status                         
Не женат / не замужем   2796  0.097639
в разводе               1189  0.070648
вдовец / вдова           951  0.066246
гражданский брак        4134  0.093130
женат / замужем        12261  0.075606

**Вывод:** 

Из полученных данных можно сделать вывод, что самый большой процент людей с просрочками по кредитами это - "Не женат / не замужем", скорее всего это связано с большими потребностями для жизни и низким уровнем дохода. 

Наименьший процент просрочки кредитов у  людей - "вдовец / вдова" и "в разводе", скорее всего это связано с тем, что у людей из это категории уже потребности восполнены или кредит был взял на бывшего супруга/супругу.

Наибольшее количество кредитов у людей из категории "женат / замужем"

### Зависимость между уровнем дохода и возвратом кредита в срок

In [23]:
data_grouped_income = data.groupby('total_income_category').agg({'debt': ['count','mean']})
data_grouped_income

debt          
                       count      mean
total_income_category                 
A                         25  0.080000
B                       5014  0.070602
C                      15921  0.084982
D                        349  0.060172
E                         22  0.090909

**Вывод:** 

Из полученных данных можно сделать вывод, что наибольший процент кредитов это люди с доходом больше 200000 и меньше 1000000 категория - С. Люди из категории - B, на втором месте по количеству кредитов, однако процент просрочки у них меньше.

Все остальные категории практически не берут кредитов или данных недостаточно. 

### Зависимость между целью кредита и возвратом кредита в срок

In [24]:
data_grouped = data.groupby('purpose_category').agg({'debt': ['count','mean']})
data_grouped

debt          
                          count      mean
purpose_category                         
операции с автомобилем     4279  0.093480
операции с недвижимостью  10751  0.072551
получение образования      3988  0.092528
проведение свадьбы         2313  0.079118

**Вывод:** 

Из полученных данных можно сделать вывод, что наибольший процент просрочки это люди с кредитом на автомобиль и образование. 

Наименьший процент просрочки это кредит на недвижимость и в этой категории наибольшее количество кредитов.

## Общий вывод

**Вывод:** 

Была представлена модель кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

На этапе предобработки данных были удалены пропуски и заменены медианым значением для каждой из категорий. Обработаны аномальные значения, заменен вещественный тип данных в столбце total_income на целочисленный, обработаны неявные дубликаты, выполнена категоризация данных по назначению кредита.

На этапе исследования данных выявлялась зависимость между различными показателями (Количество детей, семейное положение, уровнем дохода, цель кредита) и возврата кредита в срок.

Предпочтительный потенциальный заемщик это категория "женат / замужем", без детей, с уровнем дохода выше 1000000 и кредитом на недвижимость.